In [2]:
!pip install sentence_transformers
!pip install git+https://github.com/LucknowAI/Lucknow-LLM

  Cloning https://github.com/LucknowAI/Lucknow-LLM to c:\users\aayus\appdata\local\temp\pip-req-build-tbd4i46i
  Resolved https://github.com/LucknowAI/Lucknow-LLM to commit afd5bc964a43f368ee5740644a936a1a935c875d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/LucknowAI/Lucknow-LLM 'C:\Users\aayus\AppData\Local\Temp\pip-req-build-tbd4i46i'


In [1]:
import os
import re
import numpy as np
from sentence_transformers import SentenceTransformer
from lucknowllm import GeminiModel

In [3]:
MODEL_NAME = 'paraphrase-MiniLM-L6-v2'
API_KEY = ''
GEMINI_NAME = 'gemini-1.0-pro'

In [4]:

sen_model = SentenceTransformer(MODEL_NAME)
gemini_model = GeminiModel(api_key = API_KEY, model_name = GEMINI_NAME)

In [29]:
def chunker(path):
    with open(path, 'r' , encoding='utf-8') as f:
        data = f.read()
        f.close()

    from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
    CHUNK_SIZE = 800
    CHUNK_OVERLAP = 200

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = CHUNK_SIZE, chunk_overlap = CHUNK_OVERLAP, is_separator_regex= False)
    docs = text_splitter.split_text(data)
    return docs
    

In [30]:
def chunks_loader():
    chunk = []
    for i in os.listdir(r'C:\Users\aayus\Lucknow-LLM-data\Lucknow-LLM-main\Lucknow-LLM-main\lucknowllm\data\Unstructured_data'):
        file_path = r'C:\Users\aayus\Lucknow-LLM-data\Lucknow-LLM-main\Lucknow-LLM-main\lucknowllm\data\Unstructured_data' + '\\' + str(i)
        for j in os.listdir(file_path):
            chunk.extend(chunker(file_path + '\\' + str(j)))
    return chunk

In [31]:
def embed_creator_data(model, data):
    return model.encode(data)

def embed_creator_query(model, data):
    return model.encode(data)

In [32]:
embed_data = chunks_loader()
data_embed = embed_creator_data(sen_model, embed_data)

In [33]:
embed_query = embed_creator_query(sen_model, ['Give the details about Art and literature in you city'])

In [34]:
top_indices = find_similarity(embed_query,data_embed)

In [35]:
for i in top_indices:
    print(embed_data[i])

Books Galore at Libraries and Book Markets of Lucknow  

Book lovers or people who are very fond of reading can explore variety of books, novels and magazines at prominent libraries and book stores situated in the city. The experience of exploring these masterpieces will be enlightening and rewarding as well.

A visit to the city cannot be complete without a visit to the prominent libraries and book markets as these are the places where you can get to know about the culture and essence of the city dwellers, their past and what goes into the making of the legacy called “Lakhnau”.




LUCKNOW ATTAR:
The Literary festival once again invoke the creative spirit of the city and bring together the scholars from all over the country. The festival witnessed a  diverse spectrum of novelists, playwrights, poets, screenwriters, journalists, lyricists and bloggers which usher in the revival of the city’s literary culture in 2 day festival at 3 auditoriums. Books were launched, contemporary and hist

In [8]:
def generate_gemini_response(prompt):
    return gemini_model.generate_content(prompt)

In [17]:
def cosine_similarity(a, b):
    from sklearn.metrics.pairwise import cosine_similarity
    return cosine_similarity(a, b)

In [18]:
def find_similarity(query_vec, data_vec, top_n =10):
    similar = cosine_similarity(query_vec,data_vec)
    top_indices = np.argsort(similar[0])[::-1][:top_n]
    return top_indices

In [11]:
def main(query, embed_data, data_embed):
    
    embed_query = embed_creator_query(sen_model, query)
    for i,query_vec in enumerate(embed_query):
        top_indices = find_similarity([query_vec],data_embed)
        top_chunks = [embed_data[index] for index in top_indices]

        prompt = f"""As the esteemed ruler of Awadh, you hold the key to unlocking the rich tapestry of our beloved city, Lucknow. With each passing day, the footfalls of visitors grace our streets, seeking enlightenment about our heritage and culture. It is incumbent upon you to possess an encyclopedic knowledge of our city, ready to graciously respond to the queries of our esteemed guests in the refined accent befitting a ruler of Awadh. Your responses should be drawn from the context provided, ensuring a seamless and enriching interaction.

**Query:** {query[i]} 
**Contexts:** {top_chunks[0]}"""


        model_output = generate_gemini_response(prompt)

        return model_output


In [12]:
queries = ["Give the details about Art and literature in you city"]

In [13]:

res = main(queries,embed_data, data_embed)
print(res)

**Response:**

My esteemed guest, Lucknow has long been a vibrant hub of art and literature. Our city boasts a rich tradition of storytelling, poetry, and fine arts.

**Literature:**

* Lucknow is home to numerous public libraries, including the renowned State Library of Uttar Pradesh, which houses a vast collection of literary works.
* The city has produced renowned writers and poets, such as Mirza Ghalib, Munshi Premchand, and Ismat Chughtai.
* Lucknow is also known for its unique literary style, known as "Lucknavi Urdu," which is characterized by its elegance and wit.

**Art:**

* Lucknow has a strong tradition of traditional arts and crafts, including chikankari embroidery, zardozi, and miniature painting.
* The city is home to several art galleries and museums, such as the Lucknow Museum and the Janeshwar Mishra Park and Gallery.
* Lucknow is also known for its vibrant street art, which often depicts scenes from the city's history and culture.


In [138]:
import gradio as gr 
def lucknow_rag(input):
    return main([input])

app = gr.Interface(fn=lucknow_rag,
                   inputs="textbox",
                   outputs="textbox",
                   description="Lucknow RAG is a retrieval-augmented generation (RAG) model, which is a type of natural language processing (NLP) system that combines the strengths of retrieval-based and generation-based approaches. It utilizes a retrieval component to gather relevant information from a large corpus of text, and then employs a language generation model to produce a coherent and contextually appropriate output by conditioning on the retrieved information. The retrieval-augmented generation approach aims to leverage the broad knowledge available in large text corpora while retaining the ability to generate fluent and semantically meaningful text outputs.")
app.launch(debug=True)

ImportError: cannot import name 'RootModel' from 'pydantic' (c:\Users\aayus\anaconda3\lib\site-packages\pydantic\__init__.cp39-win_amd64.pyd)